# Fetching Stock Prices

In [1]:
import pandas as pd
import yfinance as yf
import datetime
from joblib import Parallel, delayed
import multiprocessing
import json 
import json.decoder
import os 
import requests

In [2]:
tmp = pd.read_csv("stocks.csv",sep=";")



In [3]:
countries_to_exclude = ["India","China","South Korea", 
                        "Hong Kong", "Malaysia", "Taiwan", 
                        "Indonesia", "Thailand", "Singapore",
                        "Argentina", "Brazil", "Venezuela",
                        "Israel", "Qatar", "Russia", "Turkey",
                        "Australia", "New Zealand"]
for i in countries_to_exclude:
    tmp = tmp[tmp["Country"] != i]


In [4]:
stock_list = tmp["Ticker"]
today = datetime.date.today()
last_week = today - datetime.timedelta(days=7)
two_years = today - datetime.timedelta(days=729)

In [9]:
def processInput(ticker,time):

    if os.path.isfile(f"finished_{time}.txt"):
        pass
    else:
        with open(f"finished_{time}.txt","w") as infile:
            infile.write("\n")
    with open(f"finished_{time}.txt","r") as infile:
        if ticker in infile.read().split("\n"):
            return
    try: 
        count = 0 
        # print(ticker)
        # for year in years:
        #     for month in range(len(months)):
                # if (year,months[month]) in hour_json[ticker]:
                #     continue
                # else:
        print(f"{ticker}")
        ticker_download = yf.download(tickers=f"{ticker}", start=f"{two_years}",end=f"{today}",interval=time,auto_adjust=True,threads=True)
        # print(ticker_download)
        # tmp = ticker_download.index.to_list()[-1][:4]
        # print(tmp)
        # print(ticker_download)
        # if tmp == "2021":
        ticker_download.to_csv(f"./data/stock_data_{time}/{ticker}_data.csv",sep=";")
        with open("finished.txt","a") as finished_file:
            finished_file.write(f"{ticker}\n")
        # hour_json[ticker].append((year,months[month]))
    except requests.exceptions.ConnectionError:
        processInput(ticker)
    except json.decoder.JSONDecodeError:
        return 
    except ValueError:
        return 
    except IndexError:
        return 
    except: 
        return 

In [10]:
Parallel(n_jobs=12)(delayed(processInput)(i,"1h") for i in stock_list)
Parallel(n_jobs=12)(delayed(processInput)(i,"1m") for i in stock_list)
Parallel(n_jobs=12)(delayed(processInput)(i,"1d") for i in stock_list)


KeyboardInterrupt: 